In [60]:
import sys
import geopandas as gp
import pandas as pd
import json
from elasticsearch import Elasticsearch, helpers
from glob import glob
import numpy as np
%load_ext autotime
ES_DEV = Elasticsearch(['https://3d6a9dd50c7c49c9ab5d23b6891bc03e.us-central1.gcp.cloud.es.io:9243'], 
                    http_auth=('elastic', 'WMzYk5RXyzE7MRShwPVwHzPX'), timeout=30)


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.28 ms (started: 2022-02-25 11:03:16 -05:00)


In [3]:
SF52 = { '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', 
                      '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', 
                      '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', 
                      '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', 
                      '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', 
                      '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', 
                      '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', 
                      '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', 
                      '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', 
                      '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', 
                      '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', 
                      '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', 
                      '54': 'WV', '55': 'WI', '56': 'WY', '72': 'PR'}
SF52R = {value : key for (key, value) in SF52.items()}

time: 767 µs (started: 2022-02-24 11:59:29 -05:00)


# ANALYZE /parcels fields for MDU

In [63]:
# elasticsearch-py scan https://elasticsearch-py.readthedocs.io/en/master/helpers.html#scan
def download_ES(index_name, es_instance = ES_DEV):
    print('Start downloading ES index ', index_name)
    match_query = {
        "query": {"match_all": {}},
    }
    res_gen = helpers.scan(es_instance, query= match_query, index= index_name) # 3 x faster than (match_all)
    df = pd.DataFrame([record['_source'] for record in res_gen])
    print('Completed downloading ES index ', index_name)
    return df
    

time: 554 µs (started: 2022-02-25 11:03:36 -05:00)


In [64]:
sf = '29'
abbrv = SF52[sf]
ABBRV = abbrv.upper()
ABBRV

state_df = download_ES(f'parcels_{sf}')

Start downloading ES index  parcels_29
Completed downloading ES index  parcels_29
time: 1min 16s (started: 2022-02-25 11:03:37 -05:00)


In [67]:
state_df.shape

(3215105, 14)

time: 1.79 ms (started: 2022-02-25 11:11:01 -05:00)


In [66]:
state_df.groupby('landuse').size()

landuse
agri       171822
commer      75043
infra        9699
rem       1561890
resi      1396651
dtype: int64

time: 558 ms (started: 2022-02-25 11:05:29 -05:00)


In [78]:
state_df.loc[state_df.dpv_type == 'S'].sample(10)

,pid,addr,lat,lon,activity,function,structure,dpv,dpv_type,rdi,building_count,footprint,GEOID,landuse
1841390,b8644b67-e471-471f-be15-f643734859d1,226 NE CHATEAU DR,39.042649,-94.259932,1100,1100,1100,V,S,Y,1,2275.0,290950149043018,resi
3096690,94d495ac-f916-4399-88fd-73f38ed8ecd0,3 HOSPITAL DR,38.938233,-92.329462,0,0,0,V,S,N,1,23203.0,290190022002000,rem
1409297,cc94d262-a4ed-49a2-85ad-c0d3f066f2af,1057 PATRICIA ST,37.322325,-89.565557,0,0,0,V,S,Y,1,1846.0,290318812001064,rem
1244047,86f12f57-075e-4ec6-8079-624f13e18850,116 MARY HAMMACK ST,38.607577,-91.222294,0,0,0,V,S,Y,1,1634.0,290718004012018,rem
1141838,551111e4-51fe-43a2-b818-bd69a5474725,8429 N TRACY AVE,39.247549,-94.564248,0,0,0,V,S,Y,1,1525.0,290470212051032,rem
297772,59a77880-c5b1-4032-ae03-07003b8b54a9,5 BRIGHTWELL CT,38.797616,-90.318739,1100,1100,1100,V,S,Y,1,1545.0,291892112021001,resi
2285693,51c86c95-1bfb-4ae3-8bc8-af41f71407d6,44 ARBORSHADOW CT,38.767549,-90.657008,1100,1100,0,V,S,Y,1,2392.0,291833113223001,resi
2878899,751c8a63-2ea9-497a-aa55-a3674707cb2d,3005 KENWOOD AVE,39.685285,-91.391579,1000,1000,0,V,S,Y,0,0.0,291734702002017,resi
806365,7f49ddd7-2c9f-42f9-a07a-0ac4d1f05a4b,29 FOURFIELD DR,38.981984,-91.013439,1100,1100,0,V,S,Y,1,2575.0,291138102012011,resi
1806951,24bda7ef-d8a4-4a09-a2ac-ff0c5aecffb7,7900 HEDGES AVE,38.978359,-94.45639,1100,1100,1100,V,S,Y,1,1731.0,290950128021009,resi


time: 848 ms (started: 2022-02-25 11:17:28 -05:00)


In [79]:
len(state_df.loc[state_df.dpv_type == 'S'])

2053374

time: 797 ms (started: 2022-02-25 11:17:58 -05:00)


In [83]:
dpv or dpv_status –  	USPS Delivery Point Validation
V The address is a confirmed delivery point. The primary range and secondary range (if present) are valid.
N The address is not a valid delivery point.

set(state_df.dpv)

{'', 'M', 'N', 'V'}

time: 176 ms (started: 2022-02-25 11:19:52 -05:00)


In [108]:
state_df.loc[state_df.dpv == 'M'].sample(5)

,pid,addr,lat,lon,activity,function,structure,dpv,dpv_type,rdi,building_count,footprint,GEOID,landuse
1350090,8d9fb9e0-e2ff-4f1c-85fc-f8e9fc15199b,205 WEST ST,36.872523,-89.599213,0,0,0,M,,,1,14175.0,292017812003000,rem
1242559,9443906a-7b84-4375-8eb6-afbb10c1e574,104 East Main Street,36.613398,-89.816059,0,0,0,M,,,1,763.0,291439604001029,rem
2509088,d707b544-59fd-4124-8ad4-a5cbfe77b3f7,126 W OSAGE ST,38.244145,-93.37787,0,0,0,M,S,,1,1571.0,290154603002086,rem
3207720,ff49f469-0b62-4b5c-88b5-59190ce5de5c,107 S MARKET ST,38.674738,-91.770415,0,0,0,M,S,,1,1673.0,291514901002071,rem
3214207,d35e9c3f-3af0-4026-b96e-d2ade06c4734,,38.525047,-91.868113,0,0,0,M,,,4,6827.0,291514902001165,rem


time: 156 ms (started: 2022-02-25 11:34:46 -05:00)


In [181]:
state_df.groupby('landuse').size()

landuse
agri       171822
commer      75043
infra        9699
rem       1561890
resi      1396651
dtype: int64

time: 493 ms (started: 2022-02-25 15:06:34 -05:00)


In [161]:
# condos, apartment complex
mdu_resi = state_df[(state_df.dpv_type == 'H') & ~(state_df.rdi == 'N')]
mdu_resi.shape # (54340, 14)
mdu_resi.to_csv('mdu_resi.csv', index=None)
# # Hybrid landuse (Most likely: residential living with biz on ground level)
# mdu_resi[mdu_resi.landuse == 'commer']
# mdu_commer[mdu_commer.landuse == 'resi']

time: 505 ms (started: 2022-02-25 12:15:38 -05:00)


In [179]:
# Most zero-building-count parcels are 2nd addr of a primary non-zero-building-count parcel!
mdu_resi[(mdu_resi.building_count == 0)].groupby('addr').size().sort_values()


addr
2525 MAIN ST APT 608                  1
400 W 49TH TER APT 2022               1
400 W 49TH TER APT 2032               1
400 W 49TH TER APT 2042               1
400 W 49TH TER APT 2052               1
                                   ... 
41 SCENIC CT                         16
3810 E CHERRY ST                     22
                                     33
904 NE SUNNYSIDE SCHOOL RD APT H     50
1800 E PRATHERSVILLE RD             124
Length: 956, dtype: int64

time: 5.58 ms (started: 2022-02-25 13:40:56 -05:00)


In [168]:
mdu_resi_bld = state_df[(state_df.dpv_type == 'H') & ~(state_df.rdi == 'N') & (state_df.building_count > 0)]
mdu_resi_bld.shape

(52979, 14)

time: 303 ms (started: 2022-02-25 13:31:09 -05:00)


In [164]:
mdu_resi.landuse.value_counts()

resi      42412
rem       11293
commer      525
infra       108
agri         13
Name: landuse, dtype: int64

time: 9.14 ms (started: 2022-02-25 12:18:01 -05:00)


In [166]:
                                                                            mdu_resi

,pid,addr,lat,lon,activity,function,structure,dpv,dpv_type,rdi,building_count,footprint,GEOID,landuse
9841,bd57cea9-4342-4ada-9114-71dcfc58bead,40 LIGHTHOUSE RD UNIT 5-303,38.216799,-92.640842,1100,1100,0,V,H,Y,1,250.0,291319627001150,resi
9871,3b63ecf5-d46f-4d58-a4f3-e8cabab801bc,62 KNOX RD UNIT V506,38.220774,-92.687111,1100,1100,0,V,H,Y,1,381.0,291319627001074,resi
9873,0b8b5964-3dbb-4d35-80dd-8a4d53ec170c,37 HAMPTON BAY RD UNIT 12,38.23217,-92.673328,1100,1100,0,V,H,Y,1,312.0,291319627001079,resi
9998,a88ce652-3c97-4037-832b-bd713fb16dc6,50 ARNOLD PALMER DR,38.210936,-92.601558,1100,1100,0,V,H,Y,1,271.0,291319628001193,resi
10020,db696f81-98a6-43e6-952a-6745a8592b35,40 LIGHTHOUSE RD,38.216779,-92.640866,1100,1100,0,V,H,Y,1,250.0,291319627001150,resi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3209873,ebeedf4a-3546-4f28-b03e-ca38bf314429,1221 E MAIN ST,38.474968,-91.835515,0,0,0,V,H,Y,1,6064.0,291514902003081,rem
3214567,d0aede53-247c-4b0e-83e0-4ef086b5a213,,38.466438,-91.821121,0,0,0,V,H,Y,3,12115.0,291514902001099,rem
3214776,d6038aed-0590-4af2-98d9-bfde66d0d627,,38.475334,-91.835085,0,0,0,V,H,Y,4,12887.0,291514902003081,rem
3214902,72ae5ad0-eda3-413d-8977-7375866819b8,,38.440293,-91.998009,0,0,0,V,H,Y,0,0.0,291514903001053,rem


time: 11.8 ms (started: 2022-02-25 12:42:05 -05:00)


In [165]:
mdu_commer[mdu_commer.landuse == 'resi']

,pid,addr,lat,lon,activity,function,structure,dpv,dpv_type,rdi,building_count,footprint,GEOID,landuse
125482,63c0bbb9-8f1d-408a-9cf1-c0e2667ccfe6,641 W FARM ROAD 182,37.117258,-93.301245,1000,1000,0,V,H,N,4,34218.0,290770041032002,resi
340590,680b5cc8-772f-489a-b1bd-476c7c563271,212 S MERAMEC AVE,38.647374,-90.339655,1100,1100,1200,V,H,N,1,11017.0,291892165003007,resi
341924,800263ed-e6d2-464a-81a5-a3161b20af55,2651 ELTAROSE DR APT A,38.736327,-90.260974,1100,1100,1200,V,H,N,3,10688.0,291892120022012,resi
363544,2ea76e30-59ba-4ad4-8d15-d98ce3117cae,1241 STRASSNER DR STE 1203,38.623132,-90.335561,1100,1100,1200,V,H,N,1,13680.0,291892173002039,resi
372585,df286f04-792c-4b76-8f24-3fc0ce542af4,1300 HOLGATE DR UNIT E1,38.567504,-90.507653,1100,1100,1200,V,H,N,1,6836.0,291892179422010,resi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2806315,95049e09-c1de-42d0-ba50-75165678b9f6,2401 E 32ND ST,37.055585,-94.489278,1000,1000,0,V,H,N,1,43554.0,290970105006012,resi
2806338,e79289f1-4e67-4f81-8725-b2d97874b959,1007 E 32ND ST,37.055511,-94.504295,1000,1000,0,V,H,N,1,8576.0,290970107003014,resi
2877845,e45fccf4-de1e-4462-87c9-c79e182a3b2c,17067 SUNDANCE DR,39.580338,-91.40709,1000,1000,0,V,H,N,2,11812.0,291734702001100,resi
2946602,2a81e83a-a21c-4df8-8b03-6712d3af6d0d,732 S RANGE LINE RD,37.082282,-94.477884,1000,1000,0,V,H,N,1,12710.0,290970105002000,resi


time: 14 ms (started: 2022-02-25 12:41:23 -05:00)


In [157]:
mdu_resi[mdu_resi.landuse == 'commer'].shape

(525, 14)

time: 8.76 ms (started: 2022-02-25 12:06:52 -05:00)


In [152]:
mdu_resi

,pid,addr,lat,lon,activity,function,structure,dpv,dpv_type,rdi,building_count,footprint,GEOID,landuse
9841,bd57cea9-4342-4ada-9114-71dcfc58bead,40 LIGHTHOUSE RD UNIT 5-303,38.216799,-92.640842,1100,1100,0,V,H,Y,1,250.0,291319627001150,resi
9871,3b63ecf5-d46f-4d58-a4f3-e8cabab801bc,62 KNOX RD UNIT V506,38.220774,-92.687111,1100,1100,0,V,H,Y,1,381.0,291319627001074,resi
9873,0b8b5964-3dbb-4d35-80dd-8a4d53ec170c,37 HAMPTON BAY RD UNIT 12,38.23217,-92.673328,1100,1100,0,V,H,Y,1,312.0,291319627001079,resi
9998,a88ce652-3c97-4037-832b-bd713fb16dc6,50 ARNOLD PALMER DR,38.210936,-92.601558,1100,1100,0,V,H,Y,1,271.0,291319628001193,resi
10020,db696f81-98a6-43e6-952a-6745a8592b35,40 LIGHTHOUSE RD,38.216779,-92.640866,1100,1100,0,V,H,Y,1,250.0,291319627001150,resi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3209873,ebeedf4a-3546-4f28-b03e-ca38bf314429,1221 E MAIN ST,38.474968,-91.835515,0,0,0,V,H,Y,1,6064.0,291514902003081,rem
3214567,d0aede53-247c-4b0e-83e0-4ef086b5a213,,38.466438,-91.821121,0,0,0,V,H,Y,3,12115.0,291514902001099,rem
3214776,d6038aed-0590-4af2-98d9-bfde66d0d627,,38.475334,-91.835085,0,0,0,V,H,Y,4,12887.0,291514902003081,rem
3214902,72ae5ad0-eda3-413d-8977-7375866819b8,,38.440293,-91.998009,0,0,0,V,H,Y,0,0.0,291514903001053,rem


time: 11.5 ms (started: 2022-02-25 12:01:48 -05:00)


In [144]:
mdu_resi_landuse = state_df[(state_df.dpv_type == 'H') & (state_df.dpv == 'V') & (state_df.landuse.isin(['rem','resi', 'commer']))]
mdu_resi_landuse.shape

(58756, 14)

time: 544 ms (started: 2022-02-25 11:59:12 -05:00)


In [102]:
# Street delivery single-family homes
single_homes = state_df.loc[(state_df.dpv_type == 'S') & (state_df.dpv == 'V') & (state_df.landuse.isin(['rem','resi']))]
single_homes.shape

(1865983, 14)

time: 1.13 s (started: 2022-02-25 11:32:24 -05:00)


In [142]:
single_homes[single_homes.rdi == 'N'].sample(10)

,pid,addr,lat,lon,activity,function,structure,dpv,dpv_type,rdi,building_count,footprint,GEOID,landuse
1864983,284b599a-74b6-4bfa-a94f-5511cfac6ca1,1000 E 50TH ST,39.037037,-94.574946,0,0,0,V,S,N,2,164204.0,290950074001010,rem
1323525,86f37e76-ac13-4b8c-89af-662fb89a9cf8,2201 ROARK VALLEY RD,36.65735,-93.273892,0,0,0,V,S,N,1,793.0,292134801061020,rem
2012619,7d85e207-b718-4c4a-957f-49697d0b1c33,951 SW 51 RD,38.228618,-93.775817,1100,9000,0,V,S,N,1,5524.0,290839504003191,resi
127108,2511aac3-fd34-4112-ae0c-cd9e7ae4c2fc,834 N NATIONAL AVE,37.217105,-93.275143,0,0,0,V,S,N,1,1437.0,290770008002058,rem
2969022,81d9a628-4951-4843-bcad-8526e5443bdd,102 N MAIN ST,37.177162,-94.565295,0,0,0,V,S,N,2,5829.0,290970113001033,rem
1547878,cf7a770d-05c6-40e0-a2ec-1b23b9a6170a,705 W 31ST ST,39.063262,-93.726379,0,0,0,V,S,N,3,70794.0,291070904005015,rem
2775502,a5be1bdc-d0a4-43c3-8e60-b992d7b6f4b3,1801 N ELSON ST,40.209473,-92.585272,0,0,0,V,S,N,1,12560.0,290019503001068,rem
902761,10dde1a6-3e12-431d-b717-1d586aa07c5b,210 S SPILMAN AVE,37.94166,-91.778455,0,0,0,V,S,N,0,0.0,291618908001044,rem
999342,0ac9a9cc-2aa1-4f17-acf4-33cc1d6bed19,109 LAWRENCE ST,37.933621,-90.780659,9000,0,0,V,S,N,1,11142.0,292214603001075,rem
1578310,dbd2438d-7320-4b9a-b6ee-84a0be84c564,1162 GOLDEN GOOSE DR,38.215347,-93.008435,0,0,0,V,S,N,0,0.0,290299507001112,rem


time: 113 ms (started: 2022-02-25 11:57:11 -05:00)


In [105]:
single_homes.rdi.value_counts()

Y    1817670
N      48245
          68
Name: rdi, dtype: int64

time: 57.7 ms (started: 2022-02-25 11:33:26 -05:00)


In [87]:
state_df.loc[state_df.rdi == 'Y'].shape

(1897877, 14)

time: 745 ms (started: 2022-02-25 11:25:06 -05:00)


In [68]:
Record Type	Description
F	Firm (ZIP+4 only)
G	General delivery (5-digit ZIP, ZIP+4, and carrier route products)
H	High-rise (ZIP+4 only)
M	Multi-carrier (carrier route product only)
P	PO box (5-digit ZIP, ZIP+4, and carrier route products)
R	Rural route/highway contract (5-digit ZIP, ZIP+4, and carrier route products) # MOST LIKELY: a rural single home
S	Street (5-digit ZIP, ZIP+4, and carrier route products)

set(state_df.dpv_type)

{'', 'G', 'H', 'P', 'R', 'S'}

time: 151 ms (started: 2022-02-25 11:11:39 -05:00)


In [ ]:
# MDU count
landuse: commer/resi?
dpv_type: H (dpv = Y)
rdi: Y or N?


# 

In [55]:
# # assemble state dataframe from individual county jsonl
# # CHECK NULL
# state_abbrv = 'ks'
# county_lines = glob(f"./processed-jsons/{state_abbrv}_*.json")
# state_df = []
# for jsonl in county_lines:
#     df = pd.read_json(jsonl)
#     if not df.empty:
#         state_df.append(df)
# state_df = pd.concat(state_df)
# state_df.shape, state_df.pid.isnull().sum()

# IDENTIFY incomplete
files = glob('es/new_*.csv')
len(files)
sorted(files)
sfs = []
for p in files:
    index_name = p.split('_')[1].split('.')[0]
    sf = index_name[len('bossdata'):]
    sfs.append(sf)
sfs.sort()
print(len(sfs), sfs, )

# MISSING states
print(' '.join([(SF52[k].lower()) for k in sorted(set(SF52.keys())-set(sfs))]))

52 ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']

time: 1.6 ms (started: 2022-02-24 22:11:13 -05:00)
